In [1]:
!pip install -U transformers peft datasets trl accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

# -------------------------------
# Config
# -------------------------------
MODEL_NAME = "Qwen/Qwen1.5-0.5B"
CSV_PATH = "/content/itinerary_dataset.csv"
OUTPUT_DIR = "./qwen_colab_output"
BATCH_SIZE = 2
EPOCHS = 3
LEARNING_RATE = 2e-4

# -------------------------------
# Load and Format Dataset
# -------------------------------
df = pd.read_csv(CSV_PATH)

prompts, responses = [], []
for _, row in df.iterrows():
    prompt = (
        f"Generate a {row['duration']}-day itinerary for {row['destination']} "
        f"with budget '{row['budget']}' and preferences:\n"
        f"Core: {row['core_prefs']}\n"
        f"Special: {row['special_prefs']}\n\nItinerary:"
    )
    prompts.append(prompt)
    responses.append(row["itinerary"])

dataset = Dataset.from_dict({"prompt": prompts, "response": responses})
dataset = dataset.map(lambda ex: {"text": f"{ex['prompt']}\n{ex['response']}"})
dataset = dataset.remove_columns([col for col in dataset.column_names if col != "text"])

# Split dataset
split = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split["train"]
val_dataset = split["test"]

# -------------------------------
# Load Tokenizer & Model
# -------------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    device_map="auto"
)
model = prepare_model_for_kbit_training(model)

# -------------------------------
# Apply LoRA
# -------------------------------
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# -------------------------------
# Training Setup
# -------------------------------
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    logging_steps=10,
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    warmup_ratio=0.1,
    gradient_accumulation_steps=4,
    report_to="none",
    optim="adamw_torch"
)

# -------------------------------
# Train
# -------------------------------
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=training_args
)

trainer.train()

# -------------------------------
# Save Model to Drive (Optional)
# -------------------------------
# from google.colab import drive
# drive.mount('/content/drive')
# model.save_pretrained("/content/drive/MyDrive/qwen_model")
# tokenizer.save_pretrained("/content/drive/MyDrive/qwen_model")

print("✅ Training complete!")


Map:   0%|          | 0/217 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

Tokenizing eval dataset:   0%|          | 0/22 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/22 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,1.775900
20,1.475400
30,1.305600
40,1.191900
50,1.206100
60,1.105200
70,1.079000


✅ Training complete!


In [4]:
SAVE_PATH = "/content/drive/MyDrive/qwen-itinerary-model"
model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)

('/content/drive/MyDrive/qwen-itinerary-model/tokenizer_config.json',
 '/content/drive/MyDrive/qwen-itinerary-model/special_tokens_map.json',
 '/content/drive/MyDrive/qwen-itinerary-model/vocab.json',
 '/content/drive/MyDrive/qwen-itinerary-model/merges.txt',
 '/content/drive/MyDrive/qwen-itinerary-model/added_tokens.json',
 '/content/drive/MyDrive/qwen-itinerary-model/tokenizer.json')

In [5]:
# 🔍 Inference with your fine-tuned Qwen model

def generate_itinerary(destination, duration, budget, core_prefs, special_prefs, max_new_tokens=600):
    prompt = (
        f"Generate a {duration}-day itinerary for {destination} with budget '{budget}' and preferences:\n"
        f"Core: {core_prefs}\n"
        f"Special: {special_prefs}\n\nItinerary:"
    )

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.8,
            top_p=0.95,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id
        )

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text[len(prompt):].strip()


# 🧪 Example prompt
print("🧳 Generated Itinerary:\n")
print(generate_itinerary(
    destination="Tokyo",
    duration=3,
    budget="low",
    core_prefs=["food", "history"],
    special_prefs=["local_experience", "early_riser"]
))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


🧳 Generated Itinerary:

**Day 1: Exploring the Land of Culture & Early Riser**  * **Morning (9:00 AM):** Begin your day early with a visit to Shinjuku Gyoen National Garden. This beautiful oasis offers a serene escape from the city bustle, with breathtaking views of Tokyo Skytree. Pack a picnic lunch in case you're hungry after exploring its diverse atmosphere.  Enjoy the gardens and soak in the ambiance.  Consider a guided tour focusing on Japanese traditional architecture or landscape design for an enriching experience. * **Late Morning (12:00 PM):** Explore Harajuku's vibrant street style with unique shops and street food stalls.  Grab some delicious ramen at a local Izakaya (Japanese pub) for a more authentic culinary experience. * **Afternoon (2:00 PM):** Head to Shibuya crossing for a thrilling experience – the iconic crossing is filled with stunning camera angles!  Take your time to walk along the avenue and capture the colorful scenery.  Alternatively, you could take advantage 

In [6]:
# 🧪 Example prompt
print("🧳 Generated Itinerary:\n")
print(generate_itinerary(
    destination="Hyderabad",
    duration=3,
    budget="low",
    core_prefs=["food", "history"],
    special_prefs=["local_experience", "early_riser"]
))

🧳 Generated Itinerary:

**Day 1: Exploring Telangana's Charm**  * **6:00 AM:** Begin your day with an early breakfast at a local café near the metro station. Afterward, take the metro to Chandrabhaga Bazaar (budget: "medium").   The street is packed with colorful saris, traditional sweets, and diverse snacks from different vendors.  Enjoy the bustling atmosphere amidst the vibrant colors.   * **8:00 AM:** Explore Chandra Narayan Nagar (or any nearby area) via a guided walking tour of Dhandharwad Charitable Hospital (budget: "low"). This historical institution offers a glimpse into the lives of locals in the region.  It's a hands-on experience that will enhance your understanding of history while also providing delicious vegetarian options.   * **10:00 AM:**  Head to Yelaguri Temple (this temple offers free admission).  Afterwards, explore the serene surroundings of Tapiya village, where you can enjoy a picnic lunch amid the greenery.  This will provide a relaxing break after a long jou